## Lab 8: Genetic Diversity

#### Q0: What's your name?

### Part 1. Retrieve relevant background material on the biology of your gene. 

First search NCBI's Gene database for your gene. 

Enter your query then run the for loop. Note that the records are dictionaries. Dictionaries consist of "key" : "value" pairs. Modify the loop to print out the name of your gene, its description, other aliases, summary and genomic location.

In this example, we'll use gene HBA2 (NM_000517) human alpha-Hemoglobin 2

The following code cell will give you an error when run as is. This is done on purpose to introduce you to solving common errors that may occur while retrieving records using nested dictionaries. Don't panic, we have seen this error before at the begining of the semester. Today I'll walk you through to resolve it.

In [ ]:
from Bio import Entrez# let NCBI know who you are
# let NCBI know who you are
Entrez.email = "" 
db = "gene"
# assign the RefSeq accession number (as a string!) to the variable "query" 
query = "" 

# look up your query at Entrez
h_search = Entrez.esearch(db=db, email = Entrez.email, term = query)
# read the results
srecord = Entrez.read(h_search)
# assign the primary ids to a variable res_ids
res_ids = srecord['IdList'] 

# for each id in the list
for r_id in res_ids:
    # look up the summary at Entrez
    h_summ = Entrez.esummary(db=db, id=r_id, email = Entrez.email)
    # read the results
    summ = Entrez.read(h_summ)
    # print out the summary
    print(summ)  
    # print out a visual separation
    print('\n') 

In [ ]:
print(summ)

In [ ]:
print(summ.keys())

In [ ]:
print(summ['DocumentSummarySet'].keys())

In [ ]:
print(summ['DocumentSummarySet']['DocumentSummary'])

In [ ]:
print(summ['DocumentSummarySet']['DocumentSummary'][0].keys())

In [ ]:
print(summ['DocumentSummarySet']['DocumentSummary'][0])

Notice that the Entrez.read(..) returns a list containing a dictionary. Find the key 'GenomicInfo', look at the corresponding key. It is a dictionary with keys, 'ChrAccVer' - the accession number of the chromosome your gene's record is based on, 'ChrLoc' - the chromosome your gene is located on, 'ExonCount' - the number of exons in your gene, 'ChrStop' - the position on the chromosome where the gene ends, 'ChrStart' - the position on the chromosome where the gene starts. 

Q1. Look closely at the values of 'ChrLoc', 'ChrStart', and 'ChrStop' - what types of data are these?**


Now that you know how to navigate across a nested dictionary, retrieve information on your gene.

First run the code below without making any changes, then analyze your output which is a list of keys for your record.

In [ ]:
for r_id in res_ids: # for each id in the list
    h_summ = Entrez.esummary(db=db, id=r_id, email = Entrez.email) # look up the summary at Entrez
    summ = Entrez.read(h_summ) # read the results
    
    if summ['DocumentSummarySet']['DocumentSummary'][0]['GenomicInfo'] != []: # if the value corresponding to the key is not blank
        print("Keys for GenomicInfo")
        #print out the keys of the dictionary summ[0]
        print(summ['DocumentSummarySet']['DocumentSummary'][0]['GenomicInfo'][0].keys()) 
        print("Keys for parent key")
        print(summ['DocumentSummarySet']['DocumentSummary'][0].keys())

Now you are ready to retrieve important information about your gene. Make sure that the values assigned to chromosome, starting_position, ending_position are integers.

  * Create a variable named oname to store the name of the organism  
  
  * Create a variable named chromosome to store the chromosome number where your gene is located
  
  * Create a variable named starting_position to store the location on the chromosome where your gene starts
  
  * Create a variable named ending_position to store the location on the chromosome where your gene ends

In [ ]:
# for each id in the list
for r_id in res_ids: 
    h_summ = Entrez.esummary(db=db, id=r_id, email = Entrez.email) # look up the summary at Entrez
    # read the results
    summ = Entrez.read(h_summ) 
    
    # redefine variable summ to include the nested paths, this will make your code more readable and easier to type
    
    # if the value corresponding to the key is not blank
    if summ['GenomicInfo']!= []: 
        # create a variable oname and assign it to the value corresponding to the key 'Organism'
        
        # display the value of oname
        print(oname)
        # create a variable chromosome and assign it to the value corresponding to the key 'ChrLoc' in the dictionary 'GenomicInfo'
        
        # create a variable starting_position and assign it to the value corresponding to the key 'ChrStart' in the dictionary 'GenomicInfo''
        
        # create a variable ending_position and assign it to the value corresponding to the key 'ChrStop' in the dictionary 'GenomicInfo'
        
        # display a sentence summarizing the position of the gene
        print("HBA2 in", oname, "is found on chromosome", chromosome, "from position", starting_position, "to position", ending_position)
        
# Convert variables chromosome, starting_position and ending_position to integer
chromosome = int(chromosome)
starting_position = int(starting_position)
ending_position = int(ending_position)

print("chromosome data type is ", type(chromosome))
print("starting_position data type is ", type(starting_position))
print("ending_position data type is ", type(ending_position))

Q2. How many exons does your gene have?

Part 2. Querying genetic diversity.

Download the Example_hemoglobin_blast.XML file from Blackboard and save it on the same directory as this .ipynb file. We're going to use Biopython to parse the xml file and loop over the records. We'll use a regular expression to return the accession numbers of the hits with an e-value less than a threshold you define.  

a) We'll find the accession number of each BLAST hit.  
b) We'll filter the results by their e-value.  
c) We'll put these pieces together in order to create a list of the accession numbers whose hits had an e-value less than a threshold you've defined.   
d) We'll use the list of accesion numbers to look up the genomic location of each record.

**First,** we need to know something about how information is stored in a **Biopython BLAST record**. The BLAST record classes are illustrated in section 7.4 of the Biopython tutorial, "The BLAST record class". http://biopython.org/DIST/docs/tutorial/Tutorial.html

Note that each blast record has three attributes - descriptions, alignments, multiple_alignment. We're most interested in the attributes of alignments.

**Second,** we need to know about **Regular expressions** (regex).
A very powerful method for searching and retrieving matches inside a text. We can find matches by defining a pattern when we don't know the exact characters used.   

A very useful resource on writing regular expressions in Python can be found here: https://docs.python.org/2/library/re.html. 

#### a) Finding the accession number of each hit

First we'll parse the xml file and print out the alignment titles. ***You don't need to change anything to the following code cell, just run it.***

In [ ]:
# import the xml parsing library from Biopython
from Bio.Blast import NCBIXML 

# enter the name of the xml file here
result_handle = open("Example_hemoglobin_blast.xml") 
# ask the NCBIXML module to parse the file, organize the file according to its xml tags
blast_records = NCBIXML.parse(result_handle)  

# for each record in all the BLAST records parsed by NCBIXML.parse
for blast_record in blast_records: 
    # for each alignment in blast_record's attribute alignments
    for alignment in blast_record.alignments: 
        # print the alignment's title, where .title is an attribute of alignment
        print(alignment.title) 

Notice that the first capital letters to appear in each title are the capital letters at the begining of the accession number and that there are two capital letters in a row.
Also notice that the first character after the accession number is "|".

With this information we can define regular expressions (or pattern) to parse out the accession number. We'll first use the regular expression method **re.search**. This method takes two arguments, the regular expression (pattern) and the string to be searched.  

To define the regular expression using python syntax from the URL given above:

- We want to find the first capital letter. 
To do this, we can define a character set consisting of all the capital letters in the alphabet writen as [A-Z]

- We also want to find two capital letters in a row. 
This is written as [A-Z]{2}. 

- After the two capital letters, there will be an unknown number of characters and then a | (pipe). The pipe will delimit our regular expression.

 . (dot) matches any single character except a newline character.

 \* (star) matches any number of the preceding character.
 
  We can use [A-Z]{2}.\* to match any two capital letters in a row followed by any number of any character.
  
  Another problem is that the \* operator is greedy, meaning it will match as many characters as it can until it hits the next element (in our case, the pipe). In our record there are multiple | in the title .\* seeks to match as many characters as possible and that would include all the | before the last one. We want to limit the greediness of \* so that it returns the match with the fewest number of characters.
  
  
- ? operator forces \* to match the first |
   
   This will return the minimal rather than the maximum match.


- The end of our match will have a |

  One problem is that | is a character with special meaning in regular expressions so to literally match a | we need to "escape it" by preceding it with a backslash, e.g. \\ |. 
  
Putting all these pieces together, our final regular expression query is
**"[A-Z]{2}.\*?\\|"**

Note that some DDBJ/EMBL/GenBank accession number formats begin with one capital letter (not two) - if your blast results include these accession numbers you will need to delete {2} from the above regular expression. 

***You don't need to change anything to the following code cell, just run it. As I explain each line, please make sure you undertand.***

In [ ]:
import re

# enter the name of the xml file here
result_handle = open("Example_hemoglobin_blast.xml") 
# ask the NCBIXML module to parse the file, organize the file according to its xml tags 
blast_records = NCBIXML.parse(result_handle) 

# for each blast_record in blast_records
for blast_record in blast_records: 
    # for each alignment in blast_record.alignments
    for alignment in blast_record.alignments: 
        # print the name of the hit, use the .title attribute of the alignment
        print(alignment.title) 
        find_acc = re.search("[A-Z]{2}.*?\|", alignment.title) # assign a variable called find_acc to the result of re.search using the query string derived above in the name of the hit 
        if find_acc == None: # if a match is not found in the alignment title
            # print out a warning, the alignment title
            print("Accession number not found in", alignment.title)  
            continue # if not found, don't print and continue to the next for alignment iteration        print "Start of match:", find_acc.start() # print the beginning of the match 
        # print the end of the match
        print("End of match:", find_acc.end()) 
        # use the begining and end of the match to return a slice of alignment.title, note that the end of the slice is the match end -1 because we don't actually want to return the | that follows the accession number
        print(alignment.title[find_acc.start():find_acc.end()-1]) 
        #print a newline character as a visual barrier between records
        print('\n') 

#### b) Filtering by e-value

First, you'll need to define the maximum e-value which supports an inferrence of homology. 

Q3. Define a relevant e-value threshold and provide an explanation below.

The e-value is found in the expect attribute of the hsps attribute of each alignment. How many hits in your BLAST results have e-values less than or equal to your threshold? For each blast hit you'll first ask if the e-value is less than or equal to your threshold, if so then increase a counter variable by one. Then print out the full length of the subject sequence, the length of the alignment, the number of identical positions within the alignment, the e-value, and the first 75 characters of each alignment.

In [ ]:
# enter the name of the xml file here
result_handle = open("") 
# ask the NCBIXML module to parse the file, organize the file according to its xml tags
blast_records = NCBIXML.parse(result_handle)  

# create a variable named e_value_thresh to hold the number for your minimum e-value threshold

# create a variable named count_hits to hold the count of the number of hits with e-value less than threshold


# for each blast_record in blast_records
  
    # for each alignment in blast_record.alignments
    
        # for each high scoring pair in alignment.hsps
        
            
            # if the e-value ("expect") is less than or equal to your threshold
            if hsp.expect <= e_value_thresh:
                # increment the counter, count_hits by one
                
                # display the length of the full subject sequence
                print('length:', alignment.length) 
                # display the length of the local alignment
                print('length of local alignment:', len(hsp.sbjct)) 
                # display the number of identical positions in local alignment
                print('identical positions:', hsp.identities) 
                # display the e-value
                print('e value:', hsp.expect) 
            
                # print out the first 75 characters of the aligned query
                print(hsp.query[0:75] + '...') 
                # print out the | characters representing matching positions for the first 75 characters of the alignment
                print(hsp.match[0:75] + '...') 
                # print out the first 75 characters of the aligned subject 
                print(hsp.sbjct[0:75] + '...') 
                # print a newline character as a visual barrier between records
                print('\n') 

# display the value of count_hits
print(count_hits)

#### c) Generate a list of the accession numbers of sequences containing hits with e-values less than your threshold. 

Now we'll combine the filtering by e-value with the regular expression designed above to create a list of the accession numbers of subject sequences containing hits with e-values less than or equal to the threshold set above.   

Q4. Do you think a single subject sequence could contain more than one BLAST hit? Write your hypothesis below, we will test this hypothesis later. 

In [ ]:
import re

# enter the name of the xml file here
result_handle = open("") 
# ask the NCBIXML module to parse the file, organize the file according to its xml tags 
blast_records = NCBIXML.parse(result_handle) 

# create an empty list named acc_list to hold the accession numbers of hits passing your filter
acc_list = [] 

# enter the minimum e-value threshold here
e_value_thresh =0.01  

# for each blast_record in blast_records
  
    # for each alignment in blast_record.alignments
     
        # for each high scoring pair in alignment.hsps
        
            # if the e-value, hsp.expect, is less than or equal to your threshold
            if hsp.expect <= e_value_thresh: 
                find_acc = re.search("[A-Z].*?\|", alignment.title) # assign a variable called find_acc to the result of re.search using the query string derived above in the name of the hit 
                if find_acc == None: # if a match is not found in the alignment title
                    print("Accession number not found in", alignment.title) # print out a warning, the alignment title 
                    # continue with next for hsp iteration
                    continue 
                
                # create a variable with the begining and end of the match from the title, note that the end of the slice is the match end -1 because we don't actually want to return the | that follows the accession number
                acc_id = alignment.title[find_acc.start():find_acc.end()-1]
                # if the accession number, acc_id, is not already in the list acc_list
                if acc_id not in acc_list:
                    # add the accession number to acc_list
                    acc_list.append(acc_id)

#print how many hits meet the minimum e-value threshold
print("how many hits meet the minimun e-value threshold:", len(acc_list))
#print the complete list of hits
print("the complete list of hits:", acc_list)

When the BLASTn query returns multiple hits within a species, we need to determine if those hits are sequence/splice variants or paralogs. We can do this by looking up the Genbank record of each hit, returning the genomic location of the hit, and comparing the location to that of the original BLAST query. 

But first, we need to group the accession numbers according to the organism in which they are found.

***You don't need to change anything to the following code cell, just run it. As I explain each line, please make sure you undertand.***

In [ ]:
Entrez.email = "" # let NCBI know who you are
db = "gene" # enter the database to query
organism_dict = {} # create an empty dictionary called organism_dict, organism name will be the key and a list of accession numbers of records in that organism

print("These are the values of the hits that passed the threshold: ", acc_list)

for i in acc_list: # loop over each accession number in the list acc_list
    h_search = Entrez.esearch(db=db, email = Entrez.email, term = i) # use Entrez.esearch to look up records associated with each accession number in the Gene database
    srecord = Entrez.read(h_search) # read the results of the Entrez.esearch query
    res_ids = srecord["IdList"] # get the list of primary ids from that query
    for r_id in res_ids: # for each id in the list res_ids
        h_summ = Entrez.esummary(db=db, id=r_id, email = Entrez.email) # query the Entrez db with that id
        summ = Entrez.read(h_summ) # read the result
        summ = summ['DocumentSummarySet']['DocumentSummary'][0]
        if summ['GenomicInfo'] != []: # only the records having information on the genomic position are relevant
            if summ['Organism']['ScientificName'] in organism_dict: # ask if the organism of the summary is in the dictionary organism_dict
                if i not in organism_dict[summ['Organism']['ScientificName']]:  # if the accesion number is not already associated with that organism
                    organism_dict[summ['Organism']['ScientificName']].append(i) # add the accession number i to the list of accession numbers
            else: # if the organism is not already in the dictionary organism_dict
                organism_dict[summ['Organism']['ScientificName']] = [i] # add the organism as a key to the dictionary and associate it with the accession number as a list
            
print("\nSame hits ordered by species and with genomic information:")
print(organism_dict) # display the values of the entire dictionary organism_dict

We're most interested in the organism in which the gene you're studying is found. So let's loop over the organism dictionary, organism_dict, and only where the key (organism name) is equal to your organism's we'll assign the records to a dictionary, chromosome_dict, where the chromosome number will be the key and the value will be a list of lists containing information about each gene located on that chromromse. 

***You don't need to change anything to the following code cell, just run it. As I explain each line, please make sure you undertand.***

In [ ]:
db = "gene" # the Entrez database we'll query
chromosome_dict = {} # a dictionary with keys = chromosomes and values = accession number, gene description, number of exons, chromosome accession number, begining position of gene, ending position of gene  

print(organism_dict.keys())

for i in organism_dict: # for each organism in the dictionary
    if 'Homo sapiens' in organism_dict.keys(): # if the organism is the same as the organism where your gene is found
        for j in organism_dict[i]: # for each accession number in the list of all accession numbers associated with that organism
            acc = j # create a variable named acc and assign it to the accession number
            h_search = Entrez.esearch(db=db, email = Entrez.email, term = j) # query the Entrez database with the accession number
            srecord = Entrez.read(h_search) # read the record returned
            res_ids = srecord["IdList"] # create a list of ids
            for r_id in res_ids: # for each id in list
                h_summ = Entrez.esummary(db=db, id=r_id, email = Entrez.email) # use the id to query the Entrez database
                summ = Entrez.read(h_summ) # read the result
                summ = summ['DocumentSummarySet']['DocumentSummary'][0]                
                if summ['GenomicInfo'] != []: # only the records having information on the genomic position are relevant
                    chrom_version = summ['GenomicInfo'][0]['ChrAccVer'] # create a variable chrom_version and assign it to the value corresponding to the key 'ChrAccVer' in the dictionary 'GenomicInfo'
                    gene_descrip = summ['Description'] # create a variable gene_descrip and assign it to the value corresponding to the key 'Description' in the dictionary summ
                    exon_num = int(summ['GenomicInfo'][0]['ExonCount']) # create a variable exon_num and assign it to the value corresponding to the key 'ExonCount' in the dictionary 'GenomicInfo'
                    hit_chrom_loc = summ['GenomicInfo'][0]['ChrLoc'] # create a variable hit_chrom_loc and assign it to the value corresponding to the key 'ChrLoc' in the dictionary 'GenomicInfo'
                    hit_gene_start = int(summ['GenomicInfo'][0]['ChrStart']) # create a variable hit_gene_start and assign it to the value corresponding to the key 'ChrStart' in the dictionary 'GenomicInfo'
                    hit_gene_end = int(summ['GenomicInfo'][0]['ChrStop']) # create a variable hit_gene_end and assign it to the value corresponding to the key 'ChrStop' in the dictionary 'GenomicInfo'
                    if hit_chrom_loc in chromosome_dict: # if the chromosome number is already a key in the dictionary chromosome_dict
                        # add a list containing, the accession number, description of the gene, number of exons, version of the chromosome accession number, the begining position of the hit and the end position of the hit, to the list associated with that chromosome key
                        chromosome_dict[hit_chrom_loc].append([acc, gene_descrip, exon_num, chrom_version, hit_gene_start, hit_gene_end])
                    else: # if the chromosome number is not already in the dictionary chromosome_dict 
                        chromosome_dict[hit_chrom_loc] = [[acc, gene_descrip, exon_num, chrom_version, hit_gene_start, hit_gene_end]] # add a the chromosome as a key of the dictionary with the value a list containing a list of the accession number, description of the gene, version of the chromosome accession number, the begining position of the hit and the end position of the hit

# display the value of chromosome_dict
print(chromosome_dict)

As a first pass, let's check how many chromosomes have good BLAST hits. 

In [ ]:
print("Hits were found on these chromosomes:", chromosome_dict.keys())

Next, we need to identify the locations where the hits occur on the chromosomes, keeping in mind that there may be sequence/splice variants both of your gene and of the potential paralogs.  
We'll divide each chromosome into "hit regions", and create a new "hit region" if a hit does not overlap with a previous hit. We'll check later if different regions are an artifact of different versions of the chromosome sequence.

In [ ]:
# create a dictionary hit_region_dict

# for each chromosome in chromosome_dict

    # add the chromosome as a key in the dictionary hit_region_dict, assign it the value of an empty list
    
    # and use a set so that duplicate regions are not allowed
    
    # for each accession number associated with that chromosome number in chromosome_dict
    
        # assign to the variable min_p the lowest of the two position numbers, this will not always be the first number as the gene might be on the - strand, look up the min function
        min_p = min(hit[4], hit[5])
        # assign to the variable max_p the larger of the two position numbers, look up the max funtion
        max_p = max(hit[4], hit[5])
        hit_region_dict[ch].add((min_p, max_p)) # add tuple containing a new region
                
# display the contents of hit_region_dict
print((hit_region_dict))

Next, let's see how many regions were found on each of the chromosomes.

In [ ]:
# loop over each chromosome in hit_region_dict

    # print out a message stating how many regions were identified on that chromosome
    print(len(hit_region_dict[ch]), "regions on chromosome", ch)

Now, let's sort the accession numbers of our good hits according to the region in which they're found and then print out the genes' accession numbers, descriptions, version of the chromosome sequence, the begining position of the gene on the chromosome, the ending  position of the gene on the chromosome, and the number of exons the gene has.

***You don't need to change anything to the following code cell, just run it. As I explain each line, please make sure you undertand.***

In [ ]:
# for each chromosome in hit_region_dict
for ch in hit_region_dict:
    # print the chromosome number
    print(ch)
    # for each region associated with that chromosome number in hit_region_dict
    for r in hit_region_dict[ch]:
        # print out the region
        print(r)
        if starting_position in range(r[0], r[1]) or ending_position in range(r[0], r[1]): # ask if your gene's location is within that region
            print("\t \t ***** Location of", query, "*****") # if so, print out a message saying so 
        for p in chromosome_dict[ch]: # for each region in the list of regions associated with the chromosome_dict entry for each chromosome
            # assign to the variable gene_min_p the lowest of the two position numbers for that gene
            gene_min_p = min(p[4], p[5])
            # assign to the variable gene_max_p the larger of the two position numbers for that gene
            gene_max_p = max(p[4], p[5])
            if gene_min_p in range(r[0], r[1]) or gene_max_p in range(r[0], r[1]):
                print("\t \t Gene id:", p[0], "Description:", p[1], "Version of chromosome sequence:", p[3], "Begining position:", p[4], "Ending position:", p[5], "Number of exons:", p[2])
        print('\n') # print a newline character to add a visual seperation between regions
    print('\n') # print a newline character to add a visual seperation between chromosomes

Q5. How many records occur within the same region as your gene? Based on the descriptions of these genes, do they have the same or simlar functions to that of your gene?

Q6. Do the records within the same region as your gene, have the exact same begining and ending postion as your gene? Do any of the records have a different number of exons than your gene?

Q7. Based on the descriptions of genes in other regions of the genome, do they have the same or similar functions to that of your gene? The same number of exons? 

Q8. Based on your findings, how many paralogs do you belive your gene has? Note the location within the genome where these paralogs are found (chromosome number and region).

That's all folks! 
Please save your notebook and related files and upload the notebook to Blackboard.